In [3]:
import requests
import numpy as np
from shapely.geometry import LineString, Point as ShapelyPoint
import json

# Fetch data
url = 'https://data.cityofnewyork.us/resource/52n9-sdep.json'
params = {
    '$where': 'intersects(the_geom, "POLYGON((-73.9927 40.7242, -73.9787 40.7202, -73.9756 40.7285, -73.9875 40.7325, -73.9927 40.7242))")',
    '$limit': 5000
}

response = requests.get(url, params=params)
data = response.json()

print(f"Fetched {len(data)} sidewalks")

# Direction calculation
def get_direction(p1, p2):
    """Return cardinal direction between two points"""
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = np.degrees(np.arctan2(dy, dx)) % 360
    
    if 45 <= angle < 135:
        return 'N'
    elif 135 <= angle < 225:
        return 'W'
    elif 225 <= angle < 315:
        return 'S'
    else:
        return 'E'

# Width and direction sampling
def sample_outer_ring_with_width(outer_ring, inner_ring, num_samples=180):
    """Sample outer ring and measure width to inner ring"""
    outer_line = LineString(outer_ring)
    inner_line = LineString(inner_ring)
    
    samples = []
    
    for i in range(num_samples):
        frac = i / (num_samples - 1)
        outer_pt = outer_line.interpolate(frac, normalized=True)
        
        # Find nearest point on inner line
        nearest_inner = inner_line.interpolate(inner_line.project(outer_pt))
        
        # Calculate width (distance * 2 since we're measuring from center)
        width = outer_pt.distance(nearest_inner) * 364000 * 2
        
        # Determine direction of this segment
        if i < num_samples - 1:
            next_pt = outer_line.interpolate((i+1) / (num_samples - 1), normalized=True)
            direction = get_direction([outer_pt.x, outer_pt.y], [next_pt.x, next_pt.y])
        else:
            direction = samples[-1]['direction'] if samples else 'E'
        
        samples.append({
            'point': [outer_pt.x, outer_pt.y],
            'width': width,
            'direction': direction
        })
    
    return samples

# Segment by direction
def segment_by_direction(samples):
    """Group consecutive samples by direction"""
    if not samples:
        return []
    
    segments = []
    current_segment = {
        'direction': samples[0]['direction'],
        'points': [samples[0]['point']],
        'widths': [samples[0]['width']]
    }
    
    for sample in samples[1:]:
        if sample['direction'] == current_segment['direction']:
            current_segment['points'].append(sample['point'])
            current_segment['widths'].append(sample['width'])
        else:
            if len(current_segment['points']) >= 2:
                segments.append(current_segment)
            current_segment = {
                'direction': sample['direction'],
                'points': [sample['point']],
                'widths': [sample['width']]
            }
    
    # Add final segment
    if len(current_segment['points']) >= 2:
        segments.append(current_segment)
    
    return segments

# Categorize width
def categorize_width(width):
    if width < 6:
        return 1
    elif width <= 12:
        return 2
    elif width <= 20:
        return 3
    return 4

# Process all sidewalks
features = []
processed_count = 0

for idx, sidewalk in enumerate(data):
    coords = sidewalk['the_geom']['coordinates'][0]
    outer_ring = coords[0]
    inner_ring = coords[1] if len(coords) > 1 else None
    
    if not inner_ring or len(outer_ring) < 4 or len(inner_ring) < 4:
        continue
    
    # Sample and measure widths
    samples = sample_outer_ring_with_width(outer_ring, inner_ring, num_samples=180)
    
    # Segment by direction
    segments = segment_by_direction(samples)
    
    # Create GeoJSON features
    for seg in segments:
        if len(seg['points']) < 2:
            continue
        
        min_width = min(seg['widths'])
        
        # Filter out unrealistic widths
        if min_width < 2 or min_width > 50:
            continue
        
        features.append({
            'type': 'Feature',
            'properties': {
                'width': round(min_width, 1),
                'category': categorize_width(min_width),
                'direction': seg['direction']
            },
            'geometry': {
                'type': 'LineString',
                'coordinates': seg['points']
            }
        })
    
    processed_count += 1
    if processed_count % 50 == 0:
        print(f"Processed {processed_count} sidewalks, created {len(features)} segments")

# Create GeoJSON
geojson = {
    'type': 'FeatureCollection',
    'features': features
}

print(f"\nFinal result: {len(features)} line segments from {processed_count} sidewalks")

# Save to file
with open('east_village_sidewalks.geojson', 'w') as f:
    json.dump(geojson, f)

print("Saved to east_village_sidewalks.geojson")

# Show summary
categories = {1: 0, 2: 0, 3: 0, 4: 0}
for feat in features:
    categories[feat['properties']['category']] += 1

print("\nCategory breakdown:")
print(f"  Category 1 (<6.5ft): {categories[1]}")
print(f"  Category 2 (6.5-8.5ft): {categories[2]}")
print(f"  Category 3 (8.5-10ft): {categories[3]}")
print(f"  Category 4 (>10ft): {categories[4]}")

Fetched 116 sidewalks
Processed 50 sidewalks, created 256 segments

Final result: 402 line segments from 78 sidewalks
Saved to east_village_sidewalks.geojson

Category breakdown:
  Category 1 (<6.5ft): 0
  Category 2 (6.5-8.5ft): 4
  Category 3 (8.5-10ft): 95
  Category 4 (>10ft): 303
